Example from (Neuroglancer)[https://github.com/google/neuroglancer/blob/master/python/examples/jupyter-notebook-demo.ipynb] implemented below with customly made ipywidgets.

In [13]:
import neuroglancer
import numpy as np

In [14]:
viewer = neuroglancer.Viewer()

In [15]:
with viewer.txn() as s:
  s.layers['image'] = neuroglancer.ImageLayer(source='precomputed://gs://neuroglancer-public-data/flyem_fib-25/image')
  s.layers['segmentation'] = neuroglancer.SegmentationLayer(source='precomputed://gs://neuroglancer-public-data/flyem_fib-25/ground_truth', selected_alpha=0.3)
  

In [16]:
with viewer.txn() as s:
    s.voxel_coordinates = [3000.5, 3000.5, 3000.5]

In [17]:
with viewer.txn() as s:
    s.layers['segmentation'].visible = False

Code below will ask to authorize into google.
After authorization, the dataset for this example is public, anyone can use it.
However, after reloading the kernel the following warning may pop-up. Ignore it, it's not fatal and everything will load:

```
tensorstore/kvstore/gcs/admission_queue.cc:108: Using default AdmissionQueue with limit 32
tensorstore/internal/oauth2/google_auth_provider.cc:149: Using credentials at /home/ambt/.config/gcloud/application_default_credentials.json
tensorstore/internal/oauth2/google_auth_provider.cc:156: Using OAuth2 AuthProvider
```

In [18]:
import tensorstore as ts
image_vol = await ts.open({'driver': 'neuroglancer_precomputed', 'kvstore': 'gs://neuroglancer-public-data/flyem_fib-25/image/'})
a = np.zeros((200,200,200), np.uint8)
def make_thresholded(threshold):
  a[...] = image_vol[3000:3200,3000:3200,3000:3200][...,0].read().result() > threshold
make_thresholded(110)
# This volume handle can be used to notify the viewer that the data has changed.
volume = neuroglancer.LocalVolume(
    a,
    dimensions=neuroglancer.CoordinateSpace(
        names=['x', 'y', 'z'],
        units='nm',
        scales=[8, 8, 8],
    ),
    voxel_offset=[3000, 3000, 3000])
with viewer.txn() as s:
  s.layers['overlay'] = neuroglancer.ImageLayer(
        source=volume,
      # Define a custom shader to display this mask array as red+alpha.
        shader="""
void main() {
  float v = toNormalized(getDataValue(0)) * 255.0;
  emitRGBA(vec4(v, 0.0, 0.0, v));
}
""",
    )

In [7]:
make_thresholded(100)
volume.invalidate()

In [8]:
with viewer.txn() as s:
    s.layers['segmentation'].segments.update([1752, 88847])
    s.layers['segmentation'].visible = True

In [9]:
import copy
new_state = copy.deepcopy(viewer.state)
new_state.layers['segmentation'].segments.add(10625)
viewer.set_state(new_state)

'11914cce3705422ffad824482680d5deb9ee81e7'

In [10]:
num_actions = 0
def my_action(s):
    global num_actions
    num_actions += 1
    with viewer.config_state.txn() as st:
      st.status_messages['hello'] = ('Got action %d: mouse position = %r' %
                                     (num_actions, s.mouse_voxel_coordinates))
    print('Got my-action')
    print('  Mouse position: %s' % (s.mouse_voxel_coordinates,))
    print('  Layer selected values: %s' % (s.selected_values,))
viewer.actions.add('my-action', my_action)
with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keyt'] = 'my-action'
    s.status_messages['hello'] = 'Welcome to this example'

In [11]:
with viewer.txn() as s:
    s.layout = '3d'
    print(s.projection_scale)
    s.projection_scale = 3000
    # s.projection_depth = 1200000

None


## Custom widgets

In [12]:
import ipywidgets as widgets

def get_header(text):
    return widgets.HTML("<h3>{}</h3>".format(text), layout=widgets.Layout(height='auto'))

# Widgets

# Zoom
zoom_slider = widgets.FloatSlider(
    value=1,
    min=0,
    max=25,
    step=1,
    description='Zoom:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

# X-Axis
x_location = widgets.FloatSlider(
    value=3000,
    min=1500,
    max=4500,
    step=100,
    description='X Location:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

# Y-Axis

y_location = widgets.FloatSlider(
    value=3000,
    min=1500,
    max=4500,
    step=100,
    description='Y Location:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

#Z-Axis

z_location = widgets.FloatSlider(
    value=1000,
    min=1000,
    max=10000,
    step=100,
    description='Z Location:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

# Layers

layers = [
    widgets.Checkbox(
        value=True,
        description='Image',
        disabled=False,
        indent=False
    ),
    widgets.Checkbox(
        value=True,
        description='Segmentation',
        disabled=False,
        indent=False
    ),
    widgets.Checkbox(
        value=True,
        description='Overlay',
        disabled=False,
        indent=False
    ),
]

# add widget to change layout
current_layout=widgets.ToggleButtons(
    options=["xy","yz","xz","4panel","3d"],                           
    disabled=False,
    value="3d",
    # layout=widgets.Layout(width='50%')
)

# Callback list

def change_zoom_value(change):
    with viewer.txn() as s:
        s.crossSectionScale = change['new']

def change_x_axis(change):
    with viewer.txn() as s:
        s.position[0] = change['new']
        
def change_y_axis(change):
    with viewer.txn() as s:
        s.position[1] = change['new']
        
def change_z_axis(change):
    with viewer.txn() as s:
        s.position[2] = change['new']
        
def change_layer(change):
    layer_name = change.owner.description.lower()
    with viewer.txn() as s:
        layer = [l for l in s.layers if l.name == layer_name]
        if len(layer) == 1:
            layer[0].visible = change['new']
            
def layout_observer(change):
    with viewer.txn() as s:
        s.layout = change['new']
        
# Observers
    
zoom_slider.observe(change_zoom_value, 'value')
x_location.observe(change_x_axis, 'value')
y_location.observe(change_y_axis, 'value')
z_location.observe(change_z_axis, 'value')

for child in layers:
    child.observe(change_layer, 'value')
    
current_layout.observe(layout_observer,'value')
    
# Display

htmlIframe = '<iframe src="{}" style="width:99%;height:98vh;"></iframe>'.format(str(viewer))

iframe_widget = widgets.HTML(value=htmlIframe,layout=widgets.Layout(width='100%',height='100%'))
display(widgets.HBox(
    children=[widgets.VBox([
        get_header("Navigation"),
        zoom_slider,
        x_location,
        y_location,
        z_location,
        get_header("Layers"),
        *layers,
        get_header("Layout"),
        current_layout
    ],
    layout=widgets.Layout(width='430px',height='100%')
), iframe_widget]))
